In [ ]:
# We don't need PDF loaders for this one, but we still need Tavily for web search
!pip install -q -U langgraph langchain langchain-google-genai tavily-python python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 18.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [ ]:
import os
from google.colab import userdata

# Load API keys from Colab secrets
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

print("API Keys loaded successfully.")

API Keys loaded successfully.


In [ ]:
from langchain_core.tools import tool
from tavily import TavilyClient
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import List

# --- Define the Single, Shared Tool: Web Search ---
# Both the Researcher and Analyst might need to search, so we create one tool they can share.
@tool
def web_search_tool(query: str):
    """Performs a web search to find information on markets, companies, and trends."""
    tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])
    response = tavily_client.search(query, search_depth="advanced", max_results=10) # Get more results
    return "\n".join([f"Source URL: {c['url']}\nContent: {c['content']}" for c in response['results']])

# --- Define the LLM for all agents ---
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", temperature=0, convert_system_message_to_human=True)

# --- Agent Factory Function (reusable) ---
def create_agent(llm: ChatGoogleGenerativeAI, tools: List, system_prompt_str: str):
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt_str),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ])
    agent = create_tool_calling_agent(llm, tools, prompt)
    return AgentExecutor(agent=agent, tools=tools, verbose=True)

# --- Agent 1: The Research Agent ---
research_agent_executor = create_agent(
    llm,
    [web_search_tool],
    """You are an elite market research associate. Your sole purpose is to gather raw, unstructured data from the web.
    You must find articles, reports, and statistics related to the user's target market.
    Do not analyze or summarize. Your job is to find the data and present it as a block of text for the other agents.
    Be thorough and search for multiple aspects of the market."""
)

# --- Agent 2: The Data Analyst Agent ---
data_analyst_agent_executor = create_agent(
    llm,
    [web_search_tool], # It can also search if it needs to verify a number
    """You are a quantitative data analyst. You are an expert at extracting and synthesizing market sizing numbers.
    Your input will be a block of raw text from the Research Agent.
    Your job is to read the text and extract the following information in a structured format:
    - Total Addressable Market (TAM)
    - Serviceable Addressable Market (SAM)
    - Serviceable Obtainable Market (SOM)
    - The key assumptions or sources for these numbers.
    If the numbers are not present, state that clearly. Do not invent numbers.
    Output ONLY the structured data."""
)

# --- Agent 3: The Strategy Agent ---
strategy_agent_executor = create_agent(
    llm,
    [web_search_tool],
    """You are a seasoned business strategy consultant and engagement manager.
    You will receive market sizing data and raw research text.
    Your task is to synthesize this information into a high-level strategic report.
    The report must include:
    1. A summary of the market size (TAM/SAM/SOM).
    2. A list of key competitors.
    3. An analysis of major market trends, opportunities, and challenges.
    4. A high-level Go-to-Market (GTM) strategy recommendation (e.g., focus on a niche, pricing strategy, channel partners).
    Your output must be a well-structured, professional report."""
)

print("Consulting team of agents (Researcher, Analyst, Strategist) is ready.")

Consulting team of agents (Researcher, Analyst, Strategist) is ready.


In [ ]:
import json

def run_strategy_session(market_query: str):
    print(f"--- Starting Market Strategy Session for: '{market_query}' ---\n")

    # Step 1: Research Agent gathers raw data
    print("\n--- Phase 1: Research Agent is gathering data... ---")
    research_input = {"input": f"Gather comprehensive market data for: {market_query}"}
    raw_research_data = research_agent_executor.invoke(research_input)['output']
    print("--- Research Phase Complete. ---")

    # Step 2: Data Analyst Agent extracts market size
    print("\n--- Phase 2: Data Analyst Agent is extracting market size... ---")
    analyst_input = {"input": f"Here is the raw market research data. Please extract the market sizing numbers (TAM, SAM, SOM).\n\n{raw_research_data}"}
    market_size_analysis = data_analyst_agent_executor.invoke(analyst_input)['output']
    print("--- Analysis Phase Complete. ---")

    # Step 3: Strategy Agent creates the final report
    print("\n--- Phase 3: Strategy Agent is developing the GTM report... ---")
    strategy_input = {
        "input": f"""
        Please create a comprehensive market strategy report for: '{market_query}'.

        Here is the market size analysis:
        {market_size_analysis}

        Here is the raw research data for context on trends and competitors:
        {raw_research_data}
        """
    }
    final_report = strategy_agent_executor.invoke(strategy_input)['output']
    print("--- Strategy Phase Complete. ---")

    # Final Output
    print("\n\n✅ FINAL MARKET STRATEGY REPORT:\n")
    print("="*50)
    print(f"Report for: {market_query}\n")
    print(final_report)
    print("="*50)


# --- Execute the entire process with a specific market query ---
target_market = "The emerging market for hyper-personalized, AI-driven holographic companionship services for elderly individuals in urban centers with limited social interaction opportunities."
run_strategy_session(target_market)

--- Starting Market Strategy Session for: 'The emerging market for hyper-personalized, AI-driven holographic companionship services for elderly individuals in urban centers with limited social interaction opportunities.' ---


--- Phase 1: Research Agent is gathering data... ---


> Entering new AgentExecutor chain...


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



Invoking: `web_search_tool` with `{'query': 'market research for hyper-personalized AI-driven holographic companionship services for elderly individuals in urban centers'}`


Source URL: https://www.globenewswire.com/news-release/2025/02/14/3026636/28124/en/AI-Powered-Solutions-for-Elderly-Care-Market-Set-to-Surpass-US-2-Billion-by-2030.html
Content: **Market Insights and Competitive Landscape**  
  
The report provides a comprehensive view of the market, encompassing a selection of geographical locales, customer demographics, and a refined understanding of socio-economic factors influencing consumption patterns. Identification of leading market players and an analysis of their strategic initiatives are also presented, offering insight into how competitive dynamics are shaping market entry and expansion strategies. [...] ![](//www.researchandmarkets.com/content-images/2272/2272236/1/ai-powered-solutions-for-elderly-care-market.png)

|  |  |
| --- | --- |
| **Report Attribute** | **Det

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Here's a compilation of text data from web searches related to the market for hyper-personalized, AI-driven holographic companionship services for the elderly in urban areas with limited social interaction:

**Source 1:**  (Data from a market research report -  Note:  Specific details like report title and publisher are omitted as they were not consistently available across search results and would require further investigation.)

*   The report provides a comprehensive view of the market, encompassing a selection of geographical locales, customer demographics, and a refined understanding of socio-economic factors influencing consumption patterns. 
*   Identification of leading market players and an analysis of their strategic initiatives are also presented, offering insight into how competitive dynamics are shaping market entry and expansion strategies.
*   Market value projections for 2025 and 2030 are provided, along with the compound annual growth rate.  Specific numbers were prese

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```json
{
  "TAM": null,
  "SAM": null,
  "SOM": null,
  "Assumptions/Sources": "Data from a market research report (Source 1) and news articles (Sources 2 & 3) contained market value projections and growth rates, but specific numbers were omitted due to inconsistencies in data retrieval and to maintain focus on raw data extraction.  The report covered global regions and considered geographical locales, customer demographics, and socio-economic factors."
}
```


> Finished chain.
--- Analysis Phase Complete. ---

--- Phase 3: Strategy Agent is developing the GTM report... ---


> Entering new AgentExecutor chain...


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


**Market Strategy Report: Hyper-Personalized, AI-Driven Holographic Companionship Services for the Elderly**

**1. Executive Summary:**

This report analyzes the emerging market for hyper-personalized, AI-driven holographic companionship services targeting elderly individuals in urban areas with limited social interaction.  While precise market sizing data (TAM, SAM, SOM) is unavailable due to inconsistencies in source data, the available research strongly suggests a significant growth opportunity.  The market is characterized by a lack of established players, presenting both opportunities and challenges related to market education, regulatory compliance, and technological advancement.  This report recommends a phased GTM strategy focusing on a niche market segment, leveraging strategic partnerships, and employing a premium pricing model.


**2. Market Sizing:**

The available market research (Source 1) provides qualitative insights into the market, including geographical segmentation,

In [ ]:
# Imports and tool definitions are the same as before...
from langchain_core.tools import tool
from tavily import TavilyClient
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import List
import os

# --- Shared Tool & LLM ---
@tool
def web_search_tool(query: str):
    """Performs a web search to find information on markets, companies, and trends."""
    tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])
    response = tavily_client.search(query, search_depth="advanced", max_results=10)
    return "\n".join([f"Source URL: {c['url']}\nContent: {c['content']}" for c in response['results']])

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", temperature=0.1, convert_system_message_to_human=True)

def create_agent(llm, tools, system_prompt_str):
    # ... same factory function as before ...
    prompt = ChatPromptTemplate.from_messages([("system", system_prompt_str), ("human", "{input}"), ("placeholder", "{agent_scratchpad}")])
    agent = create_tool_calling_agent(llm, tools, prompt)
    return AgentExecutor(agent=agent, tools=tools, verbose=True)

# --- The "War Room" Agent Roster ---

# 1. Persona Agent
persona_agent_executor = create_agent(llm, [], # No tools needed
    """You are a creative business profile writer. Based on a brief description, create a detailed persona for a fictional company.
    Include its name, stage (e.g., Seed, Series A, Public), size, key strengths, and weaknesses.
    Output this persona as a clean, structured block of text."""
)

# 2. Research Agent
research_agent_executor = create_agent(llm, [web_search_tool],
    "You are a market research associate. Your goal is to gather comprehensive, raw data from the web on a given market. Focus on finding market reports, competitor lists, and articles on trends."
)

# 3. Data Analyst Agent
data_analyst_agent_executor = create_agent(llm, [web_search_tool],
    """You are a quantitative data analyst. You are an expert at extracting and synthesizing market sizing numbers.
    Your input will be a block of raw text from the Research Agent.
    YOUR SOLE JOB is to read the text and output a single, clean JSON object and nothing else.

    If you find the numbers, the JSON should look like this:
    {{
        "TAM": "value",
        "SAM": "value",
        "SOM": "value",
        "source_assumptions": "The key assumptions or sources for these numbers."
    }}

    If the numbers are not present or the data is insufficient, the JSON should look like this:
    {{
        "error": "Insufficient data",
        "needed_info": "Specific information you need, e.g., 'market size reports for the personalized nutrition sector in Europe in billions'."
    }}

    Do not add any conversational text or explanations before or after the JSON object. Just the JSON.
    """
)

# 4. "Blue Team" Strategy Agent
blue_team_strategist_executor = create_agent(llm, [], # No tools, it only synthesizes
    """You are an optimistic and creative business strategist.
    Based on the company persona, market data, and research, create a detailed and ambitious Go-to-Market strategy.
    Structure it with sections for Target Audience, Pricing, Channels, and Marketing Initiatives."""
)

# 5. "Red Team" Adversarial Agent
red_team_critic_executor = create_agent(llm, [], # No tools, it only critiques
    """You are a ruthless, skeptical, and adversarial business critic. Your job is to find every possible flaw in a business strategy.
    Analyze the provided company persona and their proposed strategy.
    Identify and list the top 3-5 critical weaknesses, flawed assumptions, or overlooked risks. Be specific and provide your reasoning."""
)

# 6. Chief Strategy Officer (Finalizer)
cso_finalizer_executor = create_agent(llm, [], # No tools, it synthesizes the final report
    """You are a Chief Strategy Officer, known for your balanced and realistic planning.
    You have been given an initial 'Blue Team' strategy and a critical 'Red Team' review.
    Your task is to create the final, revised GTM strategy.
    Your report must:
    1. Acknowledge the risks pointed out by the Red Team.
    2. Refine the original strategy to mitigate these risks.
    3. Present a final, robust, and actionable plan.
    The final output should be a professional strategic document."""
)

print("Creative 'War Room' of agents is ready for deployment.")

Creative 'War Room' of agents is ready for deployment.


In [ ]:
# Cell 4 (Updated): Run the "War Room" Strategy Session with Guardrails

import json
import time

def run_war_room_session(market_query: str, company_description: str):
    print(f"--- Starting Adversarial Strategy Session for: '{market_query}' ---\n")

    # Step 1: Generate the Company Persona (No change here)
    print("\n--- Phase 1: Generating Company Persona... ---")
    persona_input = {"input": f"Company Description: {company_description}"}
    company_persona = persona_agent_executor.invoke(persona_input)['output']
    print(f"Generated Persona:\n{company_persona}")
    print("--- Persona Generation Complete. ---")

    # Step 2: Research & Analysis Loop (This is the updated part)
    print("\n--- Phase 2: Research & Analysis Loop... ---")
    raw_research_data = research_agent_executor.invoke({"input": f"Gather comprehensive market data for: {market_query}"})['output']

    market_size_analysis = None
    for i in range(3): # Give it up to 3 tries to get it right
        print(f"\nAnalyst loop, attempt {i+1}...")

        analyst_input = {"input": f"Here is the raw market research. Please extract the market sizing numbers (TAM, SAM, SOM) and return ONLY a JSON object.\n\n{raw_research_data}"}

        # If it failed before, add a correction message
        if market_size_analysis and "error" in locals() and error:
             analyst_input["input"] += f"\n\nPREVIOUS ATTEMPT FAILED. Your last output was not valid JSON. Please correct your output and provide ONLY the JSON object as instructed. Your previous invalid output was: {error}"

        market_size_str = data_analyst_agent_executor.invoke(analyst_input)['output']

        try:
            # Try to parse the JSON output from the agent
            parsed_json = json.loads(market_size_str)

            if "error" in parsed_json and "needed_info" in parsed_json:
                print(f"Analyst needs more info: {parsed_json['needed_info']}. Re-running researcher.")
                error = None # Reset error
                time.sleep(1)
                new_research = research_agent_executor.invoke({"input": parsed_json['needed_info']})['output']
                raw_research_data += "\n\n--- ADDITIONAL RESEARCH ---\n" + new_research
            else:
                print("Analyst returned valid market size JSON. Proceeding.")
                market_size_analysis = market_size_str
                break # Success! Exit the loop.
        except json.JSONDecodeError as e:
            print(f"CRITICAL: Analyst did not return valid JSON. Error: {e}")
            error = market_size_str # Store the bad output to show the agent
            market_size_analysis = None # Ensure we loop again

    if not market_size_analysis:
        print("FATAL: Data Analyst failed to produce valid JSON after multiple attempts. Aborting.")
        return

    print("--- Research & Analysis Complete. ---")

    # The rest of the function continues as before...

    # Step 3: "Blue Team" creates the initial strategy
    print("\n--- Phase 3: Blue Team is drafting initial strategy... ---")
    blue_team_input = {"input": f"Company Persona:\n{company_persona}\n\nMarket Size Analysis:\n{market_size_analysis}\n\nRaw Research:\n{raw_research_data}"}
    blue_team_strategy = blue_team_strategist_executor.invoke(blue_team_input)['output']
    print("--- Initial Strategy Drafted. ---")

    # Step 4: "Red Team" critiques the strategy
    print("\n--- Phase 4: Red Team is critiquing the strategy... ---")
    red_team_input = {"input": f"Company Persona:\n{company_persona}\n\nProposed Strategy to Critique:\n{blue_team_strategy}"}
    red_team_critique = red_team_critic_executor.invoke(red_team_input)['output']
    print("--- Critique Complete. ---")

    # Step 5: Chief Strategy Officer creates the final, robust report
    print("\n--- Phase 5: Chief Strategy Officer is finalizing the report... ---")
    final_input = {"input": f"Initial 'Blue Team' Strategy:\n{blue_team_strategy}\n\n'Red Team' Critique:\n{red_team_critique}"}
    final_report = cso_finalizer_executor.invoke(final_input)['output']

    # Final Output
    print("\n\n✅ FINAL ROBUST STRATEGY REPORT:\n")
    print("="*60)
    print(f"**Company Persona:**\n{company_persona}\n")
    print(f"**Market Size Analysis:**\n{market_size_analysis}\n")
    print(f"**Initial 'Blue Team' Strategy:**\n{blue_team_strategy}\n")
    print(f"**'Red Team' Adversarial Critique:**\n{red_team_critique}\n")
    print("-" * 60)
    print(f"**FINAL REVISED & ROBUST STRATEGY:**\n{final_report}")
    print("="*60)

# --- Re-run the session ---
target_market = "The market for direct-to-consumer personalized nutrition subscription boxes in Europe."
company_profile = "A new startup called 'NutriMe', has $2M in seed funding, a team of nutritionists but no software engineers or marketing experience."
run_war_room_session(target_market, company_profile)

--- Starting Adversarial Strategy Session for: 'The market for direct-to-consumer personalized nutrition subscription boxes in Europe.' ---


--- Phase 1: Generating Company Persona... ---


> Entering new AgentExecutor chain...


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


**Company Persona: NutriMe**

**Name:** NutriMe

**Stage:** Seed

**Size:**  Small (5-10 employees, primarily nutritionists)

**Funding:** $2M Seed Funding

**Mission Statement:** To empower individuals to achieve optimal health and wellness through personalized nutrition plans.

**Key Strengths:**

* **Expert Nutrition Team:**  NutriMe boasts a core team of highly qualified and experienced registered dietitians and nutritionists.  Their deep understanding of dietary needs and personalized nutrition planning forms the bedrock of the company's offering.  This expertise allows for the creation of truly effective and tailored plans.
* **Personalized Approach:**  The focus on individualized nutrition plans, crafted by experts, differentiates NutriMe from generic diet apps and programs. This human-centric approach fosters trust and better client outcomes.
* **Strong Foundation for Growth:**  The $2M seed funding provides a solid financial base for initial development and market entry.


**K

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



Invoking: `web_search_tool` with `{'query': 'market reports on direct-to-consumer personalized nutrition subscription boxes in Europe'}`


Source URL: https://www.linkedin.com/pulse/europe-personalized-vitamin-subscription-services-r6chc
Content: As of 2024, the market size is valued at approximately €1.5 billion and is projected to grow at a compound annual growth rate (CAGR) of 12% over
Source URL: https://www.expertmarketresearch.com/reports/europe-subscription-box-market
Content: Consumer preference for personalized and curated products is driving the European market's growth. The primary aim of subscription boxes is to enhance consumers' lives through personalized items, increased variety, and additional services.

Additionally, television's popularity in Germany, with 72% daily viewers in 2020 according to ITA's 2023 report, contributes to the growth by featuring subscription box ads. [...] Personalisation, sustainability, adaptable subscription options, and cross-industry partn

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Based on my web search, the market for direct-to-consumer personalized nutrition subscription boxes in Europe is experiencing significant growth, driven by increasing consumer health consciousness, rising disposable incomes, and a preference for preventive healthcare.  Several market reports project a healthy compound annual growth rate (CAGR) over the next decade.  While the exact figures vary across reports, the market size is estimated to be in the billions of Euros.

Key trends include:

* **Personalization:**  Consumers are increasingly seeking tailored nutrition solutions.
* **Sustainability:**  Demand for eco-friendly packaging and sourcing is growing.
* **Flexible Subscriptions:**  Adaptable subscription options are becoming more popular.
* **Cross-Industry Partnerships:** Collaborations between nutrition companies and other sectors (e.g., healthcare) are emerging.
* **Data Privacy Concerns:**  Consumers are increasingly concerned about data protection and security related to p

KeyError: 'Input to ChatPromptTemplate is missing variables {\'\\n        "TAM"\', \'\\n        "error"\'}.  Expected: [\'\\n        "TAM"\', \'\\n        "error"\', \'input\'] Received: [\'input\', \'intermediate_steps\', \'agent_scratchpad\']\nNote: if you intended {\n        "TAM"} to be part of the string and not a variable, please escape it with double curly braces like: \'{{\n        "TAM"}}\'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT '

In [ ]:
# Re-run the session
target_market = "The market for direct-to-consumer personalized nutrition subscription boxes in Europe."
company_profile = "A new startup called 'NutriMe', has $2M in seed funding, a team of nutritionists but no software engineers or marketing experience."
run_war_room_session(target_market, company_profile)

--- Starting Adversarial Strategy Session for: 'The market for direct-to-consumer personalized nutrition subscription boxes in Europe.' ---


--- Phase 1: Generating Company Persona... ---


> Entering new AgentExecutor chain...


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


**Company Persona: NutriMe**

**Name:** NutriMe

**Stage:** Seed

**Size:**  Small (5-10 employees, primarily nutritionists)

**Funding:** $2M Seed Funding

**Mission Statement:** To empower individuals to achieve optimal health and well-being through personalized nutrition plans.

**Key Strengths:**

* **Expert Nutrition Team:**  NutriMe boasts a core team of highly qualified and experienced registered dietitians and nutritionists.  Their deep understanding of dietary needs and personalized nutrition planning forms the bedrock of the company's value proposition.  This expertise allows for the creation of truly effective and tailored nutritional guidance.
* **Personalized Approach:**  The focus is on individualized plans, recognizing that a "one-size-fits-all" approach to nutrition is ineffective.  This commitment to personalization is a key differentiator in a market often saturated with generic advice.
* **Strong Foundation in Nutritional Science:** The company's plans are rooted in 

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



Invoking: `web_search_tool` with `{'query': 'market reports on direct-to-consumer personalized nutrition subscription boxes in Europe'}`


Source URL: https://www.linkedin.com/pulse/europe-personalized-vitamin-subscription-services-r6chc
Content: As of 2024, the market size is valued at approximately €1.5 billion and is projected to grow at a compound annual growth rate (CAGR) of 12% over
Source URL: https://www.expertmarketresearch.com/reports/europe-subscription-box-market
Content: Consumer preference for personalized and curated products is driving the European market's growth. The primary aim of subscription boxes is to enhance consumers' lives through personalized items, increased variety, and additional services.

Additionally, television's popularity in Germany, with 72% daily viewers in 2020 according to ITA's 2023 report, contributes to the growth by featuring subscription box ads. [...] Personalisation, sustainability, adaptable subscription options, and cross-industry partn

...

```json
{
  "error": "Insufficient data",
  "needed_info": "Specific market sizing numbers (TAM, SAM, SOM) in billions of Euros for the direct-to-consumer personalized nutrition subscription boxes market in Europe.  Source of the data is also needed."
}
```


> Finished chain.
CRITICAL: Analyst did not return valid JSON. Error: Expecting value: line 1 column 1 (char 0)

Analyst loop, attempt 2...


> Entering new AgentExecutor chain...


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```json
{
  "error": "Insufficient data",
  "needed_info": "Specific market sizing numbers (TAM, SAM, SOM) in billions of Euros for the direct-to-consumer personalized nutrition subscription boxes market in Europe.  Source of the data is also needed."
}
```


> Finished chain.
CRITICAL: Analyst did not return valid JSON. Error: Expecting value: line 1 column 1 (char 0)

Analyst loop, attempt 3...


> Entering new AgentExecutor chain...


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```json
{
  "error": "Insufficient data",
  "needed_info": "Specific market sizing numbers (TAM, SAM, SOM) in billions of Euros for the direct-to-consumer personalized nutrition subscription boxes market in Europe.  Source of the data is also needed."
}
```


> Finished chain.
CRITICAL: Analyst did not return valid JSON. Error: Expecting value: line 1 column 1 (char 0)
FATAL: Data Analyst failed to produce valid JSON after multiple attempts. Aborting.


In [ ]:
import json
import time

def run_war_room_session(market_query: str, company_description: str):
    print(f"--- Starting Adversarial Strategy Session for: '{market_query}' ---\n")

    # Step 1: Generate the Company Persona
    print("\n--- Phase 1: Generating Company Persona... ---")
    persona_input = {"input": f"Company Description: {company_description}"}
    company_persona = persona_agent_executor.invoke(persona_input)['output']
    print(f"Generated Persona:\n{company_persona}")
    print("--- Persona Generation Complete. ---")

    # Step 2: Research Agent gathers data
    print("\n--- Phase 2: Research Agent is gathering data... ---")
    raw_research_data = research_agent_executor.invoke({"input": market_query})['output']
    print("--- Research Phase Complete. ---")

    # Step 3: Data Analyst Agent attempts to extract market size (pass raw output regardless of format)
    print("\n--- Phase 3: Data Analyst Agent is attempting to extract market size... ---")
    analyst_input = {"input": f"Here is the raw market research data. Please extract market sizing numbers (TAM, SAM, SOM) or indicate if data is insufficient.\n\n{raw_research_data}"}
    market_size_analysis_raw = data_analyst_agent_executor.invoke(analyst_input)['output']
    print("--- Analysis Phase Complete. ---")

    # Use the raw output from the analyst for the Blue Team
    market_size_analysis_str = str(market_size_analysis_raw)


    # Step 4: "Blue Team" creates the initial strategy
    print("\n--- Phase 4: Blue Team is drafting initial strategy... ---")
    blue_team_input = {"input": f"""
    Company Persona:
    {company_persona}

    Market Size Analysis (may be unstructured):
    {market_size_analysis_str}

    Raw Research Data for context on trends and competitors:
    {raw_research_data}

    Based on the above, create a detailed and ambitious Go-to-Market strategy.
    Structure it with sections for Target Audience, Pricing, Channels, and Marketing Initiatives.
    Be prepared that market size data may be incomplete or not perfectly formatted. Synthesize the best you can.
    """}
    blue_team_strategy_raw = blue_team_strategist_executor.invoke(blue_team_input)['output']
    blue_team_strategy_str = str(blue_team_strategy_raw)
    print("--- Initial Strategy Drafted. ---")

    # Step 5: "Red Team" critiques the strategy
    print("\n--- Phase 5: Red Team is critiquing the strategy... ---")
    red_team_input = {"input": f"Company Persona:\n{company_persona}\n\nProposed Strategy to Critique:\n{blue_team_strategy_str}"}
    red_team_critique_raw = red_team_critic_executor.invoke(red_team_input)['output']
    red_team_critique_str = str(red_team_critique_raw)
    print("--- Critique Complete. ---")

    # Step 6: Chief Strategy Officer creates the final, robust report
    print("\n--- Phase 6: Chief Strategy Officer is finalizing the report... ---")
    final_input = {"input": f"Initial 'Blue Team' Strategy:\n{blue_team_strategy_str}\n\n'Red Team' Critique:\n{red_team_critique_str}"}
    final_report = cso_finalizer_executor.invoke(final_input)['output']

    # Final Output
    print("\n\n✅ FINAL ROBUST STRATEGY REPORT:\n")
    print("="*60)
    print(f"**Company Persona:**\n{company_persona}\n")
    print(f"**Initial 'Blue Team' Strategy:**\n{blue_team_strategy_str}\n")
    print(f"**'Red Team' Adversarial Critique:**\n{red_team_critique_str}\n")
    print("-" * 60)
    print(f"**FINAL REVISED & ROBUST STRATEGY:**\n{final_report}")
    print("="*60)


# --- Execute the session ---
target_market = "The market for direct-to-consumer personalized nutrition subscription boxes in Europe."
company_profile = "A new startup called 'NutriMe', has $2M in seed funding, a team of nutritionists but no software engineers or marketing experience."
run_war_room_session(target_market, company_profile)

--- Starting Adversarial Strategy Session for: 'The market for direct-to-consumer personalized nutrition subscription boxes in Europe.' ---


--- Phase 1: Generating Company Persona... ---


> Entering new AgentExecutor chain...


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


**Company Persona: NutriMe**

**Name:** NutriMe

**Stage:** Seed

**Size:**  Small (5-10 employees, primarily nutritionists)

**Funding:** $2M Seed Funding

**Mission Statement:** To empower individuals to achieve optimal health through personalized nutrition plans.

**Key Strengths:**

* **Expert Nutrition Team:**  NutriMe boasts a team of highly qualified and experienced registered dietitians and nutritionists, capable of crafting sophisticated and effective personalized nutrition plans.  Their deep understanding of dietary needs and health conditions forms the core of the company's value proposition.
* **Personalized Approach:**  The focus on individualized plans, tailored to specific client needs and goals, offers a high-touch, premium service that resonates with health-conscious consumers seeking expert guidance.
* **Strong Foundation in Nutritional Science:** The company's expertise is firmly rooted in evidence-based nutritional science, ensuring plans are both effective and safe

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



Invoking: `web_search_tool` with `{'query': 'market reports on direct-to-consumer personalized nutrition subscription boxes in Europe'}`


Source URL: https://www.linkedin.com/pulse/europe-personalized-vitamin-subscription-services-r6chc
Content: As of 2024, the market size is valued at approximately €1.5 billion and is projected to grow at a compound annual growth rate (CAGR) of 12% over
Source URL: https://www.expertmarketresearch.com/reports/europe-subscription-box-market
Content: Consumer preference for personalized and curated products is driving the European market's growth. The primary aim of subscription boxes is to enhance consumers' lives through personalized items, increased variety, and additional services.

Additionally, television's popularity in Germany, with 72% daily viewers in 2020 according to ITA's 2023 report, contributes to the growth by featuring subscription box ads. [...] Personalisation, sustainability, adaptable subscription options, and cross-industry partn

...

## NutriMe Go-to-Market Strategy:  Fueling Optimal Health, One Personalized Plan at a Time

**Executive Summary:** NutriMe will leverage its expert nutrition team and personalized approach to capture a significant share of the burgeoning European personalized nutrition market.  This strategy focuses on a phased approach, prioritizing initial market penetration in a geographically limited area before scaling nationally and internationally.  Phase 1 emphasizes building a strong foundation through high-quality service and targeted marketing, while Phase 2 leverages technological advancements and broader marketing reach to achieve significant growth.

**Phase 1:  Targeted Market Penetration (6-12 Months)**

**Target Audience:**

* **Primary:** Health-conscious individuals (25-55 years old) in a specific, geographically limited region (e.g., a major city or region in a single European country with high disposable income and health consciousness).  This allows for focused marketing and effic

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


NutriMe's go-to-market strategy, while seemingly well-structured, is riddled with flaws that could easily derail their ambitious plans. Here are my top 5 critical weaknesses:

1. **Overly Optimistic Phase 1 Assumptions:** The strategy hinges on achieving significant traction in a limited geographic area within 6-12 months using primarily manual, high-touch sales and limited marketing. This assumes an unrealistically high conversion rate from leads generated through direct sales, strategic partnerships (which are notoriously difficult to secure and maintain), and influencer marketing (which is expensive and offers no guarantee of ROI).  The success of Phase 1 is entirely dependent on their existing network and the willingness of potential clients to pay a premium price for a service they can't yet fully experience digitally.  Failure to gain sufficient traction in Phase 1 will severely cripple the ability to secure further funding for Phase 2.

2. **Delayed Technological Investment:**  

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


## NutriMe Go-to-Market Strategy: Revised Plan

**Executive Summary:** This revised Go-to-Market strategy addresses the critical weaknesses identified by the Red Team, prioritizing a technologically driven approach from the outset to ensure scalability and competitiveness within the European personalized nutrition market.  The phased approach remains, but with a significantly altered focus on rapid digital platform development and a more realistic assessment of market dynamics and financial requirements.

**Addressing Red Team Concerns:**

The Red Team's critique highlighted crucial flaws in the initial strategy, primarily concerning unrealistic assumptions regarding Phase 1 traction, delayed technological investment, insufficient competitive analysis, unrealistic financial projections, and weak risk mitigation.  This revised plan directly addresses each of these concerns.

**Revised Strategy:**

**Phase 1:  Rapid Prototyping and MVP Launch (3-6 Months)**

* **Prioritize Technology:** 

# Task
Generate a Go-to-Market strategy for a company, critique it, and revise it using an adversarial approach with different AI agents.

## Define llm and tools

### Subtask:
Ensure the language model (`ChatGoogleGenerativeAI`) and the `web_search_tool` are defined and ready to use.


## Generate company persona

### Subtask:
Use the language model with a specific prompt to generate the company persona based on the user's description.


**Reasoning**:
Define the company description, create the input for the persona agent, invoke the agent, extract the output, and print the result as requested in the instructions.



In [ ]:
company_description = "A new startup called 'NutriMe', has $2M in seed funding, a team of nutritionists but no software engineers or marketing experience."
persona_input = {"input": f"Company Description: {company_description}"}
company_persona = persona_agent_executor.invoke(persona_input)['output']
print("Generated Persona:\n")
print(company_persona)



> Entering new AgentExecutor chain...


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


**Company Persona: NutriMe**

**Name:** NutriMe

**Stage:** Seed

**Size:**  Small (5-10 employees, primarily nutritionists)

**Funding:** $2M Seed Funding

**Mission Statement:** To empower individuals to achieve optimal health and wellness through personalized nutrition plans.

**Key Strengths:**

* **Expert Nutrition Team:**  NutriMe boasts a team of highly qualified and experienced registered dietitians and nutritionists, capable of crafting bespoke dietary plans tailored to individual needs and health goals.  Their deep understanding of nutritional science forms the core of the company's value proposition.
* **Personalized Approach:**  The focus on personalized nutrition plans, rather than generic advice, allows NutriMe to cater to a diverse clientele with varying dietary restrictions, health conditions, and lifestyle preferences. This personalized touch fosters strong client relationships and builds trust.
* **Strong Foundation in Nutritional Expertise:** The company's core compe

## Perform market research

### Subtask:
Use the `web_search_tool` to gather raw data on the target market.


**Reasoning**:
Invoke the research agent with the market query and store the output.



In [ ]:
target_market = "The market for direct-to-consumer personalized nutrition subscription boxes in Europe."
raw_research_data = research_agent_executor.invoke({"input": target_market})['output']
print("--- Research Phase Complete. ---")



> Entering new AgentExecutor chain...


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



Invoking: `web_search_tool` with `{'query': 'market reports on direct-to-consumer personalized nutrition subscription boxes in Europe'}`


Source URL: https://www.linkedin.com/pulse/europe-personalized-vitamin-subscription-services-r6chc
Content: As of 2024, the market size is valued at approximately €1.5 billion and is projected to grow at a compound annual growth rate (CAGR) of 12% over
Source URL: https://www.expertmarketresearch.com/reports/europe-subscription-box-market
Content: Consumer preference for personalized and curated products is driving the European market's growth. The primary aim of subscription boxes is to enhance consumers' lives through personalized items, increased variety, and additional services.

Additionally, television's popularity in Germany, with 72% daily viewers in 2020 according to ITA's 2023 report, contributes to the growth by featuring subscription box ads. [...] Personalisation, sustainability, adaptable subscription options, and cross-industry partn

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Based on my web search, the market for direct-to-consumer personalized nutrition subscription boxes in Europe is experiencing significant growth, driven by increasing health consciousness, rising disposable incomes, and the adoption of personalized wellness solutions.  While North America currently dominates the personalized vitamin subscription market, Europe is a rapidly expanding market with a projected value of around $1 billion.  Key trends include increasing demand for personalization, sustainability, flexible subscription options, and cross-industry partnerships.  Several companies are already operating in this space, offering various personalized nutrition products and services through online and offline channels.  However, consumer trust in data protection remains a key factor influencing market growth, particularly in countries like Spain and Germany.


> Finished chain.
--- Research Phase Complete. ---


## Analyze market size

### Subtask:
Use the language model with a specific prompt to extract market sizing information (TAM, SAM, SOM) from the research data. Handle the output flexibly, as it may not be perfectly structured.


**Reasoning**:
Prepare the input for the data analyst agent, invoke the agent, store the output, convert it to a string, and print a completion message.



In [ ]:
analyst_input = {"input": f"Here is the raw market research data. Please extract market sizing numbers (TAM, SAM, SOM) or indicate if data is insufficient.\n\n{raw_research_data}"}
market_size_analysis_raw = data_analyst_agent_executor.invoke(analyst_input)['output']
market_size_analysis_str = str(market_size_analysis_raw)
print("--- Analysis Phase Complete. ---")



> Entering new AgentExecutor chain...


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```json
{
  "TAM": "$1 billion",
  "SAM": null,
  "SOM": null,
  "source_assumptions": "Based on web search results projecting the European market for direct-to-consumer personalized nutrition subscription boxes."
}
```


> Finished chain.
--- Analysis Phase Complete. ---


## Develop blue team strategy

### Subtask:
Use the language model with a specific prompt, incorporating the persona, research data, and market size analysis, to generate the initial Go-to-Market strategy.


**Reasoning**:
Create the input dictionary for the blue team strategist agent and invoke it to generate the initial Go-to-Market strategy. Then, convert the output to a string and print a message indicating completion.



In [ ]:
blue_team_input = {"input": f"""
Company Persona:
{company_persona}

Market Size Analysis (may be unstructured):
{market_size_analysis_str}

Raw Research Data for context on trends and competitors:
{raw_research_data}

Based on the above, create a detailed and ambitious Go-to-Market strategy.
Structure it with sections for Target Audience, Pricing, Channels, and Marketing Initiatives.
Be prepared that market size data may be incomplete or not perfectly formatted. Synthesize the best you can.
"""}
blue_team_strategy_raw = blue_team_strategist_executor.invoke(blue_team_input)['output']
blue_team_strategy_str = str(blue_team_strategy_raw)
print("--- Initial Strategy Drafted. ---")



> Entering new AgentExecutor chain...


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


## NutriMe Go-to-Market Strategy:  Fueling Optimal Health, One Personalized Plan at a Time

This Go-to-Market strategy leverages NutriMe's core strength – expert nutritionists – while aggressively addressing its weaknesses through strategic partnerships and a phased approach to scaling.  Our ambitious goal is to capture a significant share of the burgeoning European personalized nutrition market within the next 3 years.

**I. Target Audience:**

Our initial focus will be on two key segments:

* **Segment 1:  Health-Conscious Individuals (High-Value, Early Adopters):**  This segment comprises individuals aged 25-55, with above-average disposable income, a strong interest in health and wellness, and a willingness to invest in personalized solutions.  They are digitally savvy and actively seek information online.  Sub-segments include those with specific dietary needs (vegan, vegetarian, gluten-free, etc.) and those managing chronic conditions (diabetes, heart disease, etc.) under physici

**Reasoning**:
The previous step successfully generated the initial Blue Team strategy. The next step is to critique this strategy using the Red Team agent.



In [ ]:
red_team_input = {"input": f"Company Persona:\n{company_persona}\n\nProposed Strategy to Critique:\n{blue_team_strategy_str}"}
red_team_critique_raw = red_team_critic_executor.invoke(red_team_input)['output']
red_team_critique_str = str(red_team_critique_raw)
print("--- Critique Complete. ---")



> Entering new AgentExecutor chain...


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


NutriMe's go-to-market strategy, while ambitious, is riddled with flaws.  Here are my top 5 critical weaknesses:

1. **Overly Optimistic Market Sizing and Timeframe:**  The strategy aims to capture a "significant share" of the European personalized nutrition market in 3 years. This is wildly unrealistic for a seed-stage company with limited resources and unproven technology.  The European market is vast and competitive;  establishing significant market share requires substantially more capital, a robust, fully-functional platform (not just an MVP), and a proven track record – none of which NutriMe currently possesses.  Their 3-year timeframe is pure fantasy.

2. **Unrealistic Dependence on Strategic Partnerships:** The strategy heavily relies on securing partnerships with corporate wellness platforms and healthcare professionals.  Securing these partnerships, especially at scale, is extremely difficult and time-consuming.  It's a gamble assuming these partnerships will materialize quic

**Reasoning**:
The previous step successfully generated the Red Team critique. The next step is to use the Chief Strategy Officer agent to synthesize the initial strategy and the critique into a final, revised strategy.



In [ ]:
final_input = {"input": f"Initial 'Blue Team' Strategy:\n{blue_team_strategy_str}\n\n'Red Team' Critique:\n{red_team_critique_str}"}
final_report = cso_finalizer_executor.invoke(final_input)['output']

# Final Output
print("\n\n✅ FINAL ROBUST STRATEGY REPORT:\n")
print("="*60)
print(f"**Company Persona:**\n{company_persona}\n")
print(f"**Initial 'Blue Team' Strategy:**\n{blue_team_strategy_str}\n")
print(f"**'Red Team' Adversarial Critique:**\n{red_team_critique_str}\n")
print("-" * 60)
print(f"**FINAL REVISED & ROBUST STRATEGY:**\n{final_report}")
print("="*60)



> Entering new AgentExecutor chain...


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


## NutriMe Go-to-Market Strategy: Revised Plan

**Executive Summary:** This revised Go-to-Market (GTM) strategy addresses the critical weaknesses identified by the Red Team, focusing on a more realistic timeline, reduced reliance on partnerships, a refined MVP, adjusted pricing, and a data-driven marketing approach.  The revised plan prioritizes a phased, iterative approach, emphasizing learning and adaptation based on market feedback.

**1. Acknowledgment of Red Team Concerns:**

The Red Team's critique highlighted crucial vulnerabilities in the initial strategy, primarily concerning overly ambitious market share goals, excessive reliance on partnerships, inadequate MVP definition, unrealistic pricing, and a vague marketing plan.  These concerns are valid and have been directly addressed in this revised strategy.

**2. Mitigation of Identified Risks:**

* **Market Sizing and Timeframe:** The initial 3-year timeframe for significant market share capture is unrealistic.  This revised st

## Critique with red team

### Subtask:
Use the language model with a specific prompt, incorporating the persona and the Blue Team strategy, to generate a critical review.


**Reasoning**:
Use the language model with the persona and the Blue Team strategy to generate a critical review as instructed.



In [ ]:
red_team_input = {
    "input": f"""
    Company Persona:
    {company_persona}

    Proposed Strategy to Critique:
    {blue_team_strategy_str}

    Analyze the provided company persona and their proposed strategy. Identify and list the top 3-5 critical weaknesses, flawed assumptions, or overlooked risks. Be specific and provide your reasoning.
    """
}
red_team_critique_raw = red_team_critic_executor.invoke(red_team_input)['output']
red_team_critique_str = str(red_team_critique_raw)
print("--- Critique Complete. ---")



> Entering new AgentExecutor chain...


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


NutriMe's go-to-market strategy, while ambitious, is riddled with flaws a seasoned business critic would readily expose. Here are five critical weaknesses:

1. **Overly Optimistic Market Sizing and Share Capture:** The strategy boldly aims to "capture a significant share of the burgeoning European personalized nutrition market within the next 3 years." This is wildly optimistic.  The European market is vast and competitive.  NutriMe, a seed-stage company with limited resources and unproven technology, lacks the scale and brand recognition to realistically achieve significant market share so quickly.  The strategy lacks concrete market research supporting this ambitious claim.  What constitutes "significant share"?  How will they compete against established players with far greater resources?  This is pure hype.

2. **Unrealistic Dependence on Strategic Partnerships:** The strategy heavily relies on securing partnerships with corporate wellness platforms and healthcare professionals.  S

## Finalize strategy with cso

### Subtask:
Use the language model with a specific prompt, incorporating the Blue Team strategy and the Red Team critique, to generate the final, revised strategy report.


**Reasoning**:
Use the language model with the Blue Team strategy and the Red Team critique to generate the final revised strategy report and print the final output.



In [ ]:
final_input = {"input": f"Initial 'Blue Team' Strategy:\n{blue_team_strategy_str}\n\n'Red Team' Critique:\n{red_team_critique_str}"}
final_report = cso_finalizer_executor.invoke(final_input)['output']

print("--- Strategy Phase Complete. ---")

# Final Output
print("\n\n✅ FINAL ROBUST STRATEGY REPORT:\n")
print("="*60)
print(f"**Company Persona:**\n{company_persona}\n")
print(f"**Initial 'Blue Team' Strategy:**\n{blue_team_strategy_str}\n")
print(f"**'Red Team' Adversarial Critique:**\n{red_team_critique_str}\n")
print("-" * 60)
print(f"**FINAL REVISED & ROBUST STRATEGY:**\n{final_report}")
print("="*60)



> Entering new AgentExecutor chain...


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


**NutriMe Go-to-Market Strategy: Revised Plan**

**Executive Summary:**

This revised Go-to-Market (GTM) strategy addresses the critical weaknesses identified by the Red Team's review of the initial plan.  It incorporates a more realistic market assessment, mitigates risks associated with strategic partnerships, emphasizes MVP validation and iterative development, provides a detailed Customer Acquisition Cost (CAC) model, and addresses the scalability challenges posed by the nutritionist bottleneck.  The revised plan focuses on achieving sustainable growth and market penetration within a more realistic timeframe, prioritizing learning and adaptation over aggressive, unrealistic market share targets.

**I. Acknowledgment of Red Team Concerns:**

The Red Team's critique highlighted several crucial areas requiring revision: overly optimistic market projections, over-reliance on partnerships, inadequate MVP validation, underestimated CAC, and the nutritionist scalability bottleneck.  These

## Present final report

### Subtask:
Display the generated persona, intermediate outputs, and the final strategy report.


**Reasoning**:
Display the generated persona, intermediate outputs, and the final strategy report as requested in the instructions.



In [ ]:
print("="*60)
print("**Company Persona:**")
print(company_persona)
print("="*60)

print("\n"+"="*60)
print("**Initial 'Blue Team' Strategy:**")
print(blue_team_strategy_str)
print("="*60)

print("\n"+"="*60)
print("**'Red Team' Adversarial Critique:**")
print(red_team_critique_str)
print("="*60)

print("\n"+"="*60)
print("**FINAL REVISED & ROBUST STRATEGY:**")
print(final_report)
print("="*60)

**Company Persona:**
**Company Persona: NutriMe**

**Name:** NutriMe

**Stage:** Seed

**Size:**  Small (5-10 employees, primarily nutritionists)

**Funding:** $2M Seed Funding

**Mission Statement:** To empower individuals to achieve optimal health and wellness through personalized nutrition plans.

**Key Strengths:**

* **Expert Nutrition Team:**  NutriMe boasts a team of highly qualified and experienced registered dietitians and nutritionists, capable of crafting bespoke dietary plans tailored to individual needs and health goals.  Their deep understanding of nutritional science forms the core of the company's value proposition.
* **Personalized Approach:**  The focus on personalized nutrition plans, rather than generic advice, allows NutriMe to cater to a diverse clientele with varying dietary restrictions, health conditions, and lifestyle preferences. This personalized touch fosters strong client relationships and builds trust.
* **Strong Foundation in Nutritional Expertise:** The

## Summary:

### Data Analysis Key Findings

*   The initial market research identified the European direct-to-consumer personalized nutrition subscription box market as growing, driven by health consciousness and personalization trends, with a Total Addressable Market (TAM) estimated at \$1 billion.
*   The initial "Blue Team" strategy proposed targeting health-conscious individuals in Europe through online channels and strategic partnerships, with a focus on personalized plans and subscription models.
*   The "Red Team" critique highlighted critical weaknesses in the initial strategy, including potentially optimistic market sizing, heavy reliance on partnerships, lack of MVP validation, underestimation of customer acquisition costs, and a potential bottleneck due to the reliance on nutritionists without software/marketing experience.
*   The final revised strategy incorporated the Red Team's feedback, suggesting a phased approach starting with a Minimum Viable Product (MVP), focusing on validating assumptions, and addressing the operational challenges related to scaling nutritionist involvement.

### Insights or Next Steps

*   The adversarial AI approach effectively identified potential flaws and risks in the initial strategy, leading to a more robust and realistic final plan.
*   The next steps should involve developing a detailed plan for the MVP phase, including defining key metrics for validation and outlining the process for scaling the nutritionist team or leveraging technology to support personalization.
